In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections as matcoll
%matplotlib inline

In [ ]:
data_base_path = os.path.join("..", "data", "TrainingData")
subject_x_template = "subject_{}_{}__x.csv"
subject_x_time_template = "subject_{}_{}__x_time.csv"
subject_y_template = "subject_{}_{}__y.csv"
subject_y_time_template = "subject_{}_{}__y_time.csv"

In [ ]:
x_header = ["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"]
def load_and_join_data(patient_id, session_number):
    x_path = os.path.join(data_base_path, subject_x_template.format(patient_id, session_number))
    x_time_path = os.path.join(data_base_path, subject_x_time_template.format(patient_id, session_number))
    y_path = os.path.join(data_base_path, subject_y_template.format(patient_id, session_number))
    y_time_path = os.path.join(data_base_path, subject_y_time_template.format(patient_id, session_number))
    
    x = pd.read_csv(x_path, names=x_header)
    x_time = pd.read_csv(x_time_path, names=["time"])
    
    y = pd.read_csv(y_path, names=["label"])
    y_time = pd.read_csv(y_time_path, names=["time"])
    
    return pd.concat([x_time, x], axis=1), pd.concat([y_time, y], axis=1)

In [ ]:
patient_id = "008"
session_number = "01"
x, y = load_and_join_data(patient_id, session_number)

In [ ]:
def plot(df, target_class=None, acc_vis=["x", "y", "z"], gyro_vis=["x", "y", "z"], window=1000):
    acc = ["acc_x", "acc_y", "acc_z"]
    gyro = ["gyro_x", "gyro_y", "gyro_z"]
    colors = {"x": "r", "y": "g", "z":"b"}
    
    acc_axis_vis = [f"acc_{axis}" for axis in acc_vis]
    acc_colors = [colors[axis] for axis in acc_vis]
    gyro_axis_vis = [f"gyro_{axis}" for axis in gyro_vis]
    gyro_colors = [colors[axis] for axis in gyro_vis]
    
    y = df["label"].to_numpy()
    mask = np.isnan(y)
    y[mask] = -1
    if target_class is not None:
        indices = np.where(y == target_class)[0]
        start = max(0, indices[0] - window)
        end = min(indices[0]+window, len(y))
    else:
        start, end = 0, len(y)
        
    fig, ax = plt.subplots(3, 1, dpi=150)
    
    for col, color in zip(acc_axis_vis, acc_colors):
        ax[0].plot(df[col][start:end], c=color)
    plt.subplot(312)
    for col, color in zip(gyro_axis_vis, gyro_colors):
        ax[1].plot(df[col][start:end], c=color)
    plt.subplot(313)
    
    lines = [[(x, 0), (x, y[x])] for x in range(start, end)]
    linecoll = matcoll.LineCollection(lines, linewidths=0.5)
    colors = np.where(y==-1, 'r', 'b')
    ax[2].add_collection(linecoll)
    ax[2].scatter(range(start, end), y[start:end], c=colors[start:end], marker=",", linewidths=0.0)
    plt.show()


In [ ]:
plot(new_df, target_class=1, acc_vis=["x", "y", "z"], window=100)

In [ ]:
plot(new_df, target_class=2, acc_vis=["x", "y", "z"], window=200)